In [1]:
import pandas as pd
import numpy as np
import glob 
from pathlib import Path
from tqdm import tqdm
#package for getting file paths conveniently 
from webfish_tools.util import find_matching_files

In [2]:
def org_adcg_dot(dot_path, hybcycle_start=0,hybcycle_end=96,pos=0,num_z=1, output=True):
    """A function to combine the adcg dot file ouput
    Parameters
    ----------
    dot_path=path to dot folder
    hybcycle_start = hybcycle number begins
    hybcycle_end = when the hybcycle ends
    pos = the position number
    num_z = number of z
    output = boolean on whether to write or return the new df
    """
    #get paths for specific position in dots_detected file
    paths = []
    directory = Path(dot_path)
    for i in np.arange(hybcycle_start,hybcycle_end,1):
        adj_path = directory / f"HybCycle_{i}"
        csv_list = list(adj_path.glob(f'MMStack_Pos{pos}_zslice_*_channel_*.csv'))
        paths.append(csv_list)
    #flatten nested list
    pos_list = [str(item) for sublist in paths for item in sublist]
    
    #read in csvs
    df_list = []
    for path in pos_list:
        df = pd.read_csv(path)
        hybcycle = int(Path(path).parent.name.replace("HybCycle_",""))
        filename = path.split("/")[-1]
        split_filename = filename.split("_")
        zslice = int(split_filename[3])
        channel = int(split_filename[-1].replace(".csv",""))
        df["ch"] = channel
        df["z"] = zslice 
        df["hyb"] = hybcycle
        df=df[["hyb","ch","x","y","z","s","w"]]
        df.columns = ["hyb","ch","x","y","z","size","w"]
        #julia indexing starts from 1, so we need to subtract 1 for python purposes
        df[["x","y"]] = df[["x","y"]] - 1 
        df_list.append(df)
    final_df = pd.concat(df_list)
    final_df = final_df.reset_index(drop=True)
    
    if output == True:
        parent_dir = Path(path).parent.parent.parent
        outputfolder = parent_dir / "dots_comb" /f"Pos_{pos}"
        outputfolder.mkdir(parents=True,exist_ok=True)
        for z in range(num_z):
            final_df_z = final_df[final_df["z"]==z]
            output_path = outputfolder/f"locations_z_{z}.csv"
            final_df_z.to_csv(output_path)
    else:
        return final_df

In [3]:
dot_path = "/groups/CaiLab/personal/Lex/raw/Linus_10k_cleared_080918_NIH3T3/notebook_pyfiles/dots_detected/ADCG/"
org_adcg_dot(dot_path, hybcycle_start=0,hybcycle_end=79,pos=0,num_z=2,output=True)